In [1]:
def generate_github_link(username, repository, branch='main'):
    return f'https://github.com/{username}/{repository}/tree/{branch}'

# Replace these with your GitHub username, repository name, and branch name
username = 'fahmedakhanom24'
repository = 'Latest-Code-with-Dynamic-link'
branch = 'main'

github_link = generate_github_link(username, repository, branch)
github_link


'https://github.com/fahmedakhanom24/Latest-Code-with-Dynamic-link/tree/main'

In [3]:
import pandas as pd

file_path = 'adksurvey84.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)

df

,No,PONDNO,PONDNAME,DATE,LONG_NAD83DD,LAT_NAD83DD,DOC,TRUCOLOR
0,1,020039,WHISTLE POND,7/17/84,73.817367,44.567544,8.5,50.0
1,2,020042,MUD POND,7/17/84,73.913478,44.563381,11.7,90.0
2,3,020055,LAKE KUSHAQUA,7/17/84,74.103206,44.527550,5.7,35.0
3,4,020211,MUD POND,7/17/84,73.671806,44.561433,5.6,25.0
4,5,030006A,CHATEAUGAY NARROWS,7/17/84,74.006536,44.806711,4.5,25.0
...,...,...,...,...,...,...,...,...
1464,1465,050479,BLACK BROOK POND (UPPER),8/21/87,73.603461,43.967833,7.8,50.0
1465,1466,050421A,UNNAMED POND,8/24/87,73.640403,43.855617,8.6,40.0
1466,1467,050452,MUD POND,8/24/87,73.563736,43.972833,8.5,30.0
1467,1468,050464,BASS LAKE,8/24/87,73.677075,43.973667,6.0,15.0


In [5]:
# Get unique values in the 'PONDNAME' column
unique_pondnames = df['PONDNAME'].unique()

# Display the unique pond names
print(len(unique_pondnames))

954


In [6]:
# Convert the 'DATE' column to datetime format with the specified year format
df['DATE'] = pd.to_datetime(df['DATE'], format='%m/%d/%y')
df['DATE'] = df['DATE'].dt.strftime('%m/%d/%Y')  # Convert the date back to string with four-digit year

# Display the DataFrame
df

,No,PONDNO,PONDNAME,DATE,LONG_NAD83DD,LAT_NAD83DD,DOC,TRUCOLOR
0,1,020039,WHISTLE POND,07/17/1984,73.817367,44.567544,8.5,50.0
1,2,020042,MUD POND,07/17/1984,73.913478,44.563381,11.7,90.0
2,3,020055,LAKE KUSHAQUA,07/17/1984,74.103206,44.527550,5.7,35.0
3,4,020211,MUD POND,07/17/1984,73.671806,44.561433,5.6,25.0
4,5,030006A,CHATEAUGAY NARROWS,07/17/1984,74.006536,44.806711,4.5,25.0
...,...,...,...,...,...,...,...,...
1464,1465,050479,BLACK BROOK POND (UPPER),08/21/1987,73.603461,43.967833,7.8,50.0
1465,1466,050421A,UNNAMED POND,08/24/1987,73.640403,43.855617,8.6,40.0
1466,1467,050452,MUD POND,08/24/1987,73.563736,43.972833,8.5,30.0
1467,1468,050464,BASS LAKE,08/24/1987,73.677075,43.973667,6.0,15.0


In [ ]:
ee.Initialize()



# Define the Landsat 5 bands and their corresponding standard names
LC5_BANDS = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'ST_B6', 'QA_PIXEL']
STD_NAMES = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'temp', 'QA']

all_lakes=ee.FeatureCollection('projects/ee-mazarderakhsh/assets/ALAP-ALTM-Lakes-195')

# Define a function to mask clouds and cloud shadows from Landsat 5 surface reflectance imagery
def maskL457sr(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)
    waterMask= image.select('QA_PIXEL').eq(5440).Or(image.select('QA_PIXEL').eq(5504))
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBand = image.select('ST_B6').multiply(0.00341802).add(-124.15)
    return image.addBands(opticalBands, None, True).addBands(thermalBand, None, True).updateMask(qaMask).updateMask(saturationMask).updateMask(waterMask).clip(all_lakes)



# Define a function to compute the mean reflectance values for the specified bands within the region of interest (lake)
def reflectance(img, lake):
    reflectance_values = img.reduceRegion(reducer=ee.Reducer.median(), geometry=lake_polygon, scale=30).select(STD_NAMES)
    return img.set('DATE_SMP', img.date().format()).set('reflectance', reflectance_values)

# Initialize an empty list to store the dataframes for each lake
dfs = []

# Create a dictionary to map each lake name to a numerical label
lake_name_to_label = {lake_name: i for i, lake_name in enumerate(lake_names)}



# Loop through each lake name and retrieve Landsat 5 imagery for that lake
for SITE_NAME in lake_names:
    # Retrieve the lake using the filter
    lake=ee.FeatureCollection('projects/ee-mazarderakhsh/assets/ALTM_ALAP_Sta_coordinates')\
         .filter(ee.Filter.eq('Name', SITE_NAME))

   
    # Get the latitude and longitude of the station point
    lat = ee.Number(lake.first().geometry().coordinates().get(1)).getInfo()
    lon = ee.Number(lake.first().geometry().coordinates().get(0)).getInfo()

     # Create a polygon around the station point using the latitude and longitude
    #lake_polygon = create_lake_polygon(lon, lat).buffer(1000)
    lake_polygon = ee.Geometry.Point(lon, lat).buffer(100)

    # Retrieve Landsat 5 imagery for the specific lake
    l5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
        .filter(ee.Filter.calendarRange(1, 12, 'month')) \
        .filter(ee.Filter.calendarRange(1985, 1987, 'year')) \
        .filterBounds(lake_polygon) \
        .filter(ee.Filter.lt('CLOUD_COVER', 25)) \
        .map(maskL457sr) \
        .select(LC5_BANDS, STD_NAMES)

        
    #merge landsat 5 and landsat 8
    lall = ee.ImageCollection(l5.merge(l8));

    # Map the reflectance function over the Landsat 5 ImageCollection for the specific lake
    map_reflectance = lall.map(lambda img: reflectance(img, lake_polygon))

    # Reduce the mapped image collection to get reflectance values for the specific lake
    list_reflectance = map_reflectance.reduceColumns(ee.Reducer.toList(2), ['DATE_SMP', 'reflectance']).values().get(0)

    # Convert the results to a pandas DataFrame
    df_reflectance = pd.DataFrame(list_reflectance.getInfo(), columns=['DATE_SMP', 'reflectance'])
    df_reflectance['DATE_SMP'] = pd.to_datetime(df_reflectance['DATE_SMP'])
    df_reflectance['DATE_SMP'] = df_reflectance['DATE_SMP'].dt.date
    df_reflectance['reflectance'] = df_reflectance['reflectance'].apply(lambda x: {k: v for k, v in x.items() if v is not None})

    # Unpack the 'reflectance' dictionary and create separate columns for each band
    df_reflectance = pd.concat([df_reflectance.drop('reflectance', axis=1),
                                df_reflectance['reflectance'].apply(pd.Series).astype('float64', errors='ignore')], axis=1)

    # Add a new column for the lake name
    df_reflectance['SITE_NAME'] = SITE_NAME

    # Add a new column for the lake label (numeric representation)
    df_reflectance['LAKE_LABEL'] = lake_name_to_label[SITE_NAME]

    # Add the DataFrame to the list
    dfs.append(df_reflectance)


# Concatenate all DataFrames into a single DataFrame
df_all_lakes = pd.concat(dfs, ignore_index=True)

# Sort the DataFrame by 'DATE_SMP' in ascending order
df_all_lakes.sort_values(by='DATE_SMP', inplace=True)

df_all_lakes.dropna(inplace=True)

df_all_lakes

#36361 rows × 11 columns